* Applying the Sigmoid Signal on TEC Token Holders
* Checking on and off signals

In [1]:
import param as pm
import panel as pn
import holoviews as hv
import pandas as pd
import numpy as np
import hvplot.pandas


# For Pie Charts
from math import pi
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.palettes import Category20

In [2]:
df = pd.read_csv('output/TEGR1.csv')

In [3]:
class TECQFSME(pm.Parameterized):
    boost_factor = pm.Number(1, bounds=(0,4), step=0.1, doc="Multiplicative factor to apply to boosting coefficient.")
    dataset  = pm.DataFrame(columns={'amountUSD', 'projectId'}, precedence=-1, doc="Dataset of donations. Must contain amountUSD and projectId columns.")
    matching_pool = pm.Integer(25_000, bounds=(0, 100_000), step=5_000, doc="Matching pool amount.")
    total_donations = pm.Number(0, constant=True, doc="Summation of amountUSD from donations dataset.")
    total_funding_pool = pm.Number(0, constant=True, doc="Summation of matching_pool and total_donations.")
    allocations  = pm.DataFrame(precedence=-1, doc="Percentages allocation table.")
    results  = pm.DataFrame(precedence=-1, doc="Matched and unmatched funding amounts. Allocation percentages times funding amounts.")
    
    def __init__(self, **params):
        super().__init__(**params)
        self.dataset = self.qf(self.dataset)
        self.update()
        
    @staticmethod
    def qf(df, column_name='amountUSD', new_column_name='quadratic_amount'):
        """
        Takes a specefied column as the donations column. Applies the QF algorithm to produce a new specefied column and intermediate calculation columns.
        """
        df = df.copy(deep=True)
        df[f'{column_name}_allocation'] = df[column_name] / df[column_name].sum()
        df[f'sqrt({column_name})'] = np.sqrt(df[column_name])
        df[f'sum(sqrt({column_name}))'] = df.groupby('projectId')[f'sqrt({column_name})'].transform('sum')
        df[f'sq(sum(sqrt({column_name})))'] = df[f'sum(sqrt({column_name}))'].transform(lambda x: x**2)
        df[f'{new_column_name}_allocation'] = df[f'sq(sum(sqrt({column_name})))'] / df[f'sq(sum(sqrt({column_name})))'].sum()
        df[new_column_name] = df[column_name].sum() * df[f'{new_column_name}_allocation']
        return df
    
    @staticmethod
    def signal_boost_v1(df, boost_factor, boost_column='amountUSD', new_column_name='amount_boosted'):
        """
        Given a dataset and a specefied column, applies the flag boost algorithm.
        Requires that the dataset contain 'tec_token_flag' and 'tea_flag'.
        """
        df['coefficient'] = 1 + boost_factor * (df['tec_tokens_flag'].astype(int) | df['tea_flag'].astype(int))
        df[new_column_name] = df[boost_column] * df['coefficient']
        return df

    
    @pm.depends('boost_factor', 'matching_pool', watch=True)
    def update(self):
        # Update total donations and funding pool
        with pm.edit_constant(self):
            self.total_donations = self.dataset['amountUSD'].sum()
            self.total_funding_pool = self.matching_pool + self.total_donations
        
        with pm.parameterized.batch_call_watchers(self):
            # Generate and apply the signal boosting coefficient
            self.dataset = self.signal_boost_v1(self.dataset, self.boost_factor, boost_column='amountUSD', new_column_name='amount_boosted')

            # Compute the Boosted Allocation
            self.dataset = self.qf(self.dataset, column_name='amount_boosted', new_column_name='quadratic_amount_boosted')

            # Remove the intermediate steps
            self.dataset = self.dataset[self.dataset.columns[~self.dataset.columns.str.contains('sqrt')]]

            # Create an allocations table that contains allocation percentages grouped and summed by project. 
            allocation_columns = ['projectId'] + list(self.dataset.columns[self.dataset.columns.str.contains('allocation')])
            self.allocations = self.dataset[allocation_columns].groupby('projectId').sum()

            # Generate the unmatched results table by multiplying allocation percentages by total donations
            unmatched_results = self.total_donations * self.allocations
            
            # Generate the matched results table by multiplying allocation percentages by total donations plus matching pool
            matched_results = self.total_funding_pool * self.allocations
            
            # Merge matched and unmatched results
            self.results = unmatched_results.merge(matched_results, left_index=True, right_index=True, suffixes=('_unmatched', '_matched'))
            
            # Sort results by funding amount
            self.results = self.results.sort_values('quadratic_amount_allocation_matched', ascending=False)

            # Save the boosting percentage stat
            self.results['Percentage Boost'] = 100 * ((self.results['quadratic_amount_boosted_allocation_matched'] - self.results['quadratic_amount_allocation_matched']) / self.results['quadratic_amount_allocation_matched'] + 1)

        
    @pm.depends('dataset')
    def view_expertise_signal(self):
        return self.results.rename({
            'quadratic_amount_allocation_unmatched': 'QF Unmatched', 
            'quadratic_amount_allocation_matched': 'QF Matched', 
            'quadratic_amount_boosted_allocation_matched': 'QF Matched + SME',
        }, axis=1).hvplot.bar(
            y=['QF Unmatched', 'QF Matched', 'QF Matched + SME'],
            rot=45,
            stacked=False,
            title="Adding Expertise into the QF Signal",
        ).opts(multi_level=False, legend_position='top_right')
    
    @pm.depends('dataset')
    def view_percentage_boost(self):
        return self.results.hvplot.bar(
            y='Percentage Boost', 
            color='purple', 
            ylim=(0, 180), 
            yformatter="%.0f%%", 
            yticks=list(range(0,200,20)),
            grid=True,
            height=400,
            rot=45,
            title="SME Boost Funding as % of QF Funding by Project",
        ) * hv.HLine(100)
    
    @pm.depends('dataset')
    def view_sme_signal_boost_by_donation(self):
        return self.dataset.hvplot.area(
            y='amountUSD', y2='amount_boosted', 
            logy=True, color='green', title="Logarithmic SME Signal Boost by Donation", 
            xlabel='index')
            
    
    def view(self):
        return pn.Row(self.param, pn.Column(self.view_sme_signal_boost_by_donation, self.view_percentage_boost))

In [4]:
tec_qf_sme = TECQFSME(dataset=df.copy(deep=True))

In [5]:
tec_qf_sme.view()

Row
    [0] Column(margin=(5, 10), name='TECQFSME')
        [0] StaticText(value='<b>TECQFSME</b>')
        [1] FloatSlider(end=4, name='Boost factor', value=1)
        [2] IntSlider(end=100000, name='Matching pool', step=5000, value=25000)
        [3] FloatInput(description='Summation of a..., disabled=True, name='Total donations', value=6142.066124660001)
        [4] FloatInput(description='Summation of m..., disabled=True, name='Total funding pool', value=31142.06612466)
    [1] Column
        [0] ParamMethod(method, _pane=HoloViews, defer_load=False)
        [1] ParamMethod(method, _pane=HoloViews, defer_load=False)

In [6]:
dataset = tec_qf_sme.dataset

In [7]:
dataset

,id,projectId,applicationId,voter,grantAddress,amount,amountUSD,coefficient,rawScore,balance_tec,tec_tokens_flag,balance_tea,tea_flag,amountUSD_allocation,quadratic_amount_allocation,quadratic_amount,amount_boosted,amount_boosted_allocation,quadratic_amount_boosted_allocation,quadratic_amount_boosted
0,0x24a5bbf1,0x64a30a4b,19,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,2,28.57,0.0,0.0,3.0,1.0,0.001495,0.003203,19.673692,18.368665,0.001966,0.003701,34.576555
1,0x3dce13bb,0xc401c980,6,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1,27.21,0.0,0.0,0.0,0.0,0.000667,0.000174,1.067316,4.094567,0.000438,0.000160,1.496497
2,0x4cf20243,0x97589cd1,7,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1,28.57,0.0,0.0,0.0,0.0,0.012121,0.002432,14.937749,74.446665,0.007969,0.002575,24.055432
3,0x2b032f10,0xec026845,16,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1,23.56,0.0,0.0,0.0,0.0,0.000909,0.001748,10.734921,5.583500,0.000598,0.001820,17.000698
4,0x0842753b,0xa9bdf738,29,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1,23.56,0.0,0.0,0.0,0.0,0.000909,0.002944,18.083146,5.583500,0.000598,0.003195,29.844210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,0x26e1e300,0x97589cd1,7,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1,29.74,0.0,0.0,0.0,0.0,0.000301,0.002432,14.937749,1.847803,0.000198,0.002575,24.055432
253,0xa21ca1aa,0xec026845,16,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1,21.07,0.0,0.0,0.0,0.0,0.000300,0.001748,10.734921,1.843793,0.000197,0.001820,17.000698
254,0x634b5156,0xf1f4942d,24,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1,21.07,0.0,0.0,0.0,0.0,0.000300,0.001632,10.024514,1.843793,0.000197,0.001370,12.796094
255,0x4efa29aa,0xcf3165f4,10,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1,18.04,0.0,0.0,0.0,0.0,0.000300,0.000047,0.285854,1.843793,0.000197,0.000041,0.383108


In [8]:
dataset.hvplot.area(y='amountUSD', y2='amount_boosted', logy=False, color='green', title="SME Signal Boost by Donation", width=1000, height=500, ylim=(0,None), xlabel='index')

:Area   [index]   (amountUSD,amount_boosted)

In [9]:
dataset.hvplot.area(y='amountUSD', y2='amount_boosted', logy=True, color='green', title="Logarithmic SME Signal Boost by Donation", width=1000, height=500, xlabel='index')

:Area   [index]   (amountUSD,amount_boosted)

In [10]:
from functools import reduce

# Compute the difference between consecutive indices for each group
dataset['diff'] = dataset.groupby('voter').apply(lambda x: x.index.to_series().diff().fillna(0)).reset_index(level=0, drop=True)

# Use the cumulative sum of these differences as a helper series
dataset['block'] = (dataset['diff'] > 1).groupby(dataset['voter']).cumsum()

# Group by 'Category' and the helper series 'block'
grouped = dataset.groupby(['voter', 'block'])

# Initialize an empty plot
plots = []

# Iterate over each group and create an hvplot visualization
for name, group in grouped:
    plot = group.hvplot.area(label=name[0], y='amountUSD', y2='amount_boosted', logy=True, title="Logarithmic SME Signal Boost by Donation by Donor", width=1000, height=500, xlabel='index')
    plots.append(plot)
    
combined_plot = reduce(lambda x, y: x * y, plots)

dataset.hvplot.step(y='amountUSD', logy=True, color='green', title="Logarithmic SME Signal Boost by Donation Colored by Donor", width=1000, height=500, xlabel='index') * combined_plot

:Overlay
   .Curve.I              :Curve   [index]   (amountUSD)
   .Area.A_0x01d3a04a    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x04e8e5aa    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x0f465406    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x109991e3    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x114b242d    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x12137de8    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x1626255e    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x2dd2036c    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x2e2853b5    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x33878e07    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x364f0ad6    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x3b067af8    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x3e31155a    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x3e7f0545    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x410d86e3    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x4194ce73    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x4318cc44    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x434f5325.I  :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x434f5325.II :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x4405f427    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x468fd68b    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x4b2f6d3a    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x4c9f98b5    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x4f8c531d    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x5136cdfc    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x524cb61b    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x5658f4b3    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x56c246b0    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x66df5a77    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x684ce401    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x6b5918d8    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x70469299    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x732da82e    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x751747b1    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x7ab13936    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x7d547666    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x7ee9033a    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x839395e2    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x84ad6912    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x89f04f5c    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x8a4ff766    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x8b5d7121    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x8bf1c501    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x8f4ad500    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x9390fa86    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x94e9b636    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x9885a849    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x9ba96198    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x9f55abf3    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xa2a2602b    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xa2b16c27    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xab612504    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xae45a838    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xae6ef687    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xaedf9a1f    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xb08f0ca7    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xb1

In [11]:
# Compute the difference between consecutive indices for each group
dataset['diff'] = dataset.groupby('projectId').apply(lambda x: x.sort_index().index.to_series().diff().fillna(0)).reset_index(level=0, drop=True)

# Use the cumulative sum of these differences as a helper series
dataset['block'] = (dataset['diff'] > 1).groupby(dataset['projectId']).cumsum()

# Group by 'Category' and the helper series 'block'
grouped = dataset.groupby(['projectId', 'block'])

from functools import reduce

# Initialize an empty plot
plots = []

# Iterate over each group and create an hvplot visualization
for name, group in dataset.sort_values('projectId').reset_index(drop=True).groupby('projectId'):
    plot = group.hvplot.area(
        label=name[0], 
        y='amountUSD', 
        y2='amount_boosted', 
        logy=False, title="Logarithmic SME Signal Boost by Donation by Project", width=1000, height=500, xlabel='index')
    plots.append(plot)
    
combined_plot = reduce(lambda x, y: x * y, plots)

# Display the combined plot
combined_plot


:Overlay
   .Area.A_0.I    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.II   :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.III  :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.IV   :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.V    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.VI   :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.VII  :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.VIII :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.IX   :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.X    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.XI   :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.XII  :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.XIII :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.XIV  :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.XV   :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.XVI  :Area   [index]   (amountUSD,amount_boosted)

In [12]:
dataset['boosted'] = dataset['amount_boosted'] - dataset['amountUSD']

boosted = dataset[dataset['boosted'] > 0]

In [13]:
from functools import reduce

# Initialize an empty plot
plots = []

# Iterate over each group and create an hvplot visualization
for name, group in boosted.sort_values('projectId').reset_index(drop=True).groupby('projectId', as_index=False):
    plot = group.hvplot.bar(
        label=name[0], 
        y='boosted',
        x='index',
        logy=True, 
        title="Logarithmic SME Signal Boost by Donation by Project", 
        width=1000, height=500, rot=45,
        xlabel='index',
    ).opts(logy=True, ylim=(1e-1, None))
    plots.append(plot)
    
combined_plot = reduce(lambda x, y: x * y, plots)

# Display the combined plot
combined_plot


:Overlay
   .Bars.A_0.I    :Bars   [index]   (boosted)
   .Bars.A_0.II   :Bars   [index]   (boosted)
   .Bars.A_0.III  :Bars   [index]   (boosted)
   .Bars.A_0.IV   :Bars   [index]   (boosted)
   .Bars.A_0.V    :Bars   [index]   (boosted)
   .Bars.A_0.VI   :Bars   [index]   (boosted)
   .Bars.A_0.VII  :Bars   [index]   (boosted)
   .Bars.A_0.VIII :Bars   [index]   (boosted)
   .Bars.A_0.IX   :Bars   [index]   (boosted)
   .Bars.A_0.X    :Bars   [index]   (boosted)
   .Bars.A_0.XI   :Bars   [index]   (boosted)
   .Bars.A_0.XII  :Bars   [index]   (boosted)
   .Bars.A_0.XIII :Bars   [index]   (boosted)
   .Bars.A_0.XIV  :Bars   [index]   (boosted)
   .Bars.A_0.XV   :Bars   [index]   (boosted)
   .Bars.A_0.XVI  :Bars   [index]   (boosted)

In [14]:
boosted.sort_values('projectId').rename_axis('Index').reset_index(drop=False).hvplot.bar(
        label=name[0], 
        y='boosted',
        x='Index',
        logy=True, 
        title="Logarithmic SME Signal Boost by Donation by Project", 
        width=1000, height=500, rot=90,
        xlabel='index',
        color='projectId',
        cmap='Category20',
    ).opts(logy=True, ylim=(1e-1, None))

:Bars   [Index]   (boosted,projectId)

In [15]:
boosted.hvplot.bar(
        label=name[0], 
        y='boosted',
        x='index',
        logy=True, 
        title="Logarithmic SME Signal Boost by Donation by Project", 
        width=1000, height=500, rot=90,
        xlabel='index',
        color='projectId',
        cmap='Category20',
    ).opts(logy=True, ylim=(1e-1, None))

:Bars   [index]   (boosted,projectId)

In [16]:
boosted.hvplot.bar(
        label=name[0], 
        y='boosted',
        x='index',
        logy=True, 
        title="Logarithmic SME Signal Boost by Donation by Donor", 
        width=1200, height=600, rot=90,
        xlabel='index',
        color='voter',
        cmap='Category20',
    ).opts(ylim=(1e-1, None))

:Bars   [index]   (boosted,voter)

In [17]:
data = boosted.groupby('voter', sort=False)['boosted'].sum().to_frame(name='Boosted Funding')
data.reset_index().hvplot.bar(
    rot=90,
    title="Directly Boosted Funding Amounts Per Donor",
    height=740,
    width=1100,
    color='voter',
    x='voter',
    cmap='Category20',
    logy=True,
).opts(ylim=(1e-1, None))

:Bars   [voter]   (Boosted Funding)

In [18]:
data['angle'] = data['Boosted Funding']/data['Boosted Funding'].sum() * 2*pi

# Use the Category20 colormap and cycle through it if there are more than 20 categories
colors = [Category20[20][i % 20] for i in range(len(data))]
data['color'] = colors

p = figure(height=650, title="Donor Boosting Distribution", toolbar_location=None,
           tools="hover", tooltips="Donor: @voter\n Boosted Funding: $@{Boosted Funding}{1,11}", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4, 
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='voter', source=data.reset_index())

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

show(p)

In [19]:
dataset['Donation Index per Project'] = dataset.groupby('projectId').cumcount() + 1

In [20]:
dataset.hvplot.area(y='amountUSD', y2='amount_boosted',  by='projectId', x='Donation Index per Project', stacked=False, alpha=1, title="SME Signal Boost by Donation by Project", width=1000, height=500, ylim=(0,None)).opts(legend_position='top_right')#.opts(hv.opts.Area(color=hv.Palette('Category20')))

:NdOverlay   [projectId]
   :Area   [Donation Index per Project]   (amountUSD,amount_boosted)

In [21]:
dataset.hvplot.area(y='amountUSD', y2='amount_boosted', by='projectId', x='Donation Index per Project', stacked=False, alpha=1, logy=True, title="Logarithmic SME Signal Boost by Donation by Project", width=1000, height=500).opts(legend_position='top_right')#.opts(hv.opts.Area(color=hv.Palette('Category20')))

:NdOverlay   [projectId]
   :Area   [Donation Index per Project]   (amountUSD,amount_boosted)

In [22]:
dataset['Donation Index per Donor'] = dataset.groupby('voter').cumcount() + 1

In [23]:
dataset.hvplot.area(
    y='amountUSD', y2='amount_boosted',  by='voter', x='Donation Index per Donor', stacked=False, alpha=1, 
    title="SME Signal Boost by Donation by Donor",
    width=1000, height=500, ylim=(0,None)).opts(legend_position='top_right')#.opts(hv.opts.Area(color=hv.Palette('Category20')))

:NdOverlay   [voter]
   :Area   [Donation Index per Donor]   (amountUSD,amount_boosted)

In [24]:
dataset.sort_values('amount_boosted', ascending=False)

,id,projectId,applicationId,voter,grantAddress,amount,amountUSD,coefficient,rawScore,balance_tec,...,quadratic_amount,amount_boosted,amount_boosted_allocation,quadratic_amount_boosted_allocation,quadratic_amount_boosted,diff,block,boosted,Donation Index per Project,Donation Index per Donor
137,0x15ed0804,0x23387567,9,0xb08f0ca7,0xbbD107D7,3.000000e+17,565.651322,2,36.72,4483.885742,...,19.587279,1131.302644,0.121104,0.004101,38.309405,8.0,5,565.651322,6,9
143,0xf72239dc,0xdd9b885d,13,0x109991e3,0x75D923fB,5.000000e+17,936.545861,1,19.50,0.000000,...,69.150807,936.545861,0.100255,0.009894,92.429467,2.0,9,0.000000,11,1
54,0x259f62a8,0xdd9b885d,13,0x434f5325,0x75D923fB,4.200000e+20,419.712378,2,23.56,0.000000,...,69.150807,839.424756,0.089859,0.009894,92.429467,19.0,3,419.712378,4,1
129,0x74c047af,0x23387567,9,0xb08f0ca7,0xbbD107D7,2.000000e+17,379.203690,2,36.72,4483.885742,...,19.587279,758.407380,0.081186,0.004101,38.309405,28.0,4,379.203690,5,1
71,0x3318a9b6,0xdd9b885d,13,0xe594469f,0x75D923fB,4.210000e+20,419.729868,1,37.12,0.000000,...,69.150807,419.729868,0.044931,0.009894,92.429467,16.0,4,0.000000,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0x1f8ac179,0xf1f4942d,24,0xcee0b5f0,0x4f8c531d,2.000000e+14,0.368890,1,19.29,0.000000,...,10.024514,0.368890,0.000039,0.001370,12.796094,13.0,12,0.000000,13,6
244,0xefdbd106,0x64a30a4b,19,0xcee0b5f0,0xA26d6AEB,2.000000e+14,0.368890,1,19.29,0.000000,...,19.673692,0.368890,0.000039,0.003701,34.576555,16.0,15,0.000000,17,3
243,0x1c2c4893,0xcf3165f4,10,0xcee0b5f0,0x7f3eb18E,2.000000e+14,0.368890,1,19.29,0.000000,...,0.285854,0.368890,0.000039,0.000041,0.383108,77.0,3,0.000000,4,2
242,0x2e14e84d,0xa9bdf738,29,0xcee0b5f0,0x5041A1C1,2.000000e+14,0.368890,1,19.29,0.000000,...,18.083146,0.368890,0.000039,0.003195,29.844210,22.0,24,0.000000,25,1


In [25]:
dataset.hvplot.area(
    y='amountUSD', y2='amount_boosted',  by='voter', x='Donation Index per Donor', stacked=False, alpha=1, 
    title="Logarithmic SME Signal Boost by Donation by Donor", logy=True,
    width=1000, height=500, ylim=(1e-1,None)).opts(legend_position='top_right')#.opts(hv.opts.Area(color=hv.Palette('Category20')))

:NdOverlay   [voter]
   :Area   [Donation Index per Donor]   (amountUSD,amount_boosted)

In [26]:
boosted.sort_values('amount_boosted', ascending=False).head(20)[['id', 'projectId', 'applicationId', 'voter', 'grantAddress','amountUSD','coefficient','amount_boosted', 'boosted']]

,id,projectId,applicationId,voter,grantAddress,amountUSD,coefficient,amount_boosted,boosted
137,0x15ed0804,0x23387567,9,0xb08f0ca7,0xbbD107D7,565.651322,2,1131.302644,565.651322
54,0x259f62a8,0xdd9b885d,13,0x434f5325,0x75D923fB,419.712378,2,839.424756,419.712378
129,0x74c047af,0x23387567,9,0xb08f0ca7,0xbbD107D7,379.203690,2,758.407380,379.203690
38,0xaedf9bda,0xe6424ab2,22,0x839395e2,0x8110d1D0,199.679876,2,399.359752,199.679876
130,0xe3967215,0x64a30a4b,19,0xb08f0ca7,0xA26d6AEB,189.601845,2,379.203690,189.601845
131,0xcd61ec0c,0xe6424ab2,22,0xb08f0ca7,0x8110d1D0,189.601845,2,379.203690,189.601845
37,0xaf671999,0x97589cd1,7,0x839395e2,0x0035cC37,99.839938,2,199.679876,99.839938
229,0xfad3c681,0x8d6f0c7b,3,0x3e7f0545,0x808DB7C1,55.367738,2,110.735477,55.367738
45,0xf48511f9,0x64a30a4b,19,0x839395e2,0xA26d6AEB,49.919969,2,99.839938,49.919969
127,0xd0c532c8,0xec026845,16,0x7ee9033a,0x45b79C6b,38.116350,2,76.232699,38.116350


In [27]:
data = boosted.groupby('projectId')['boosted'].sum().to_frame(name='Boosted Funding').sort_values('Boosted Funding', ascending=False)

# Extract the default color cycle from hvplot
default_colors = hv.plotting.util.process_cmap('Category20',  ncolors=len(data))

# Create a color map for the bar chart using the default color cycle
color_map = {cat: default_colors[i] for i, cat in enumerate(data.index)}


data.hvplot.bar(
    rot=90,
    title="Directly Boosted Funding Amounts Per Project",
    height=540,
    width=1100,
).opts(color='projectId', cmap=color_map, legend_position='top_right')

:Bars   [projectId]   (Boosted Funding)

In [28]:
data['angle'] = data['Boosted Funding']/data['Boosted Funding'].sum() * 2*pi

# Use the Category20 colormap and cycle through it if there are more than 20 categories
colors = [Category20[20][i % 20] for i in range(len(data))]
data['color'] = colors

p = figure(height=650, title="Project Boosting Distribution", toolbar_location=None,
           tools="hover", tooltips="Project: @projectId\n Boosted Funding: $@{Boosted Funding}{1,11}", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4, 
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='projectId', source=data.reset_index())

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

show(p)

In [29]:
data = boosted.groupby('voter')['boosted'].sum().to_frame(name='Boosted Funding').sort_values('Boosted Funding', ascending=False)

# Extract the default color cycle from hvplot
default_colors = hv.plotting.util.process_cmap('Category20',  ncolors=len(data))

# Create a color map for the bar chart using the default color cycle
color_map = {cat: default_colors[i] for i, cat in enumerate(data.index)}


data.hvplot.bar(
    rot=90,
    title="Directly Boosted Funding Amounts Per Donor",
    height=740,
    width=1100,
).opts(color='voter', cmap=color_map, legend_position='top_right')

:Bars   [voter]   (Boosted Funding)

In [30]:
data['angle'] = data['Boosted Funding']/data['Boosted Funding'].sum() * 2*pi

# Use the Category20 colormap and cycle through it if there are more than 20 categories
colors = [Category20[20][i % 20] for i in range(len(data))]
data['color'] = colors

p = figure(height=650, title="Donor Boosting Distribution", toolbar_location=None,
           tools="hover", tooltips="Donor: @voter\n Boosted Funding: $@{Boosted Funding}{1,11}", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4, 
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='voter', source=data.reset_index())

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

show(p)

In [31]:
data = dataset[['amountUSD', 'boosted']].sum().to_frame(name='Value').rename_axis('Category').reset_index()
data['Category'] = ['Default Donation Amounts', 'Boost Amounts']
data

,Category,Value
0,Default Donation Amounts,6142.066125
1,Boost Amounts,3199.535370


In [32]:
data['angle'] = data['Value']/data['Value'].sum() * 2*pi

# Use the Category20 colormap and cycle through it if there are more than 20 categories
colors = [Category20[20][i % 20] for i in range(len(data))]
data['color'] = colors

p = figure(height=650, title="Donations Amounts vs. Boost Amounts", toolbar_location=None,
           tools="hover", tooltips="Category: @Category <br> Value: $@{Value}{1,11}", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4, 
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Category', source=data.reset_index())

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

show(p)

In [33]:
tec_qf_sme.allocations

,amountUSD_allocation,quadratic_amount_allocation,amount_boosted_allocation,quadratic_amount_boosted_allocation
projectId,,,,
0x10b3f00e,0.007078,0.001472,0.007227,0.001493
0x23387567,0.163461,0.035079,0.212862,0.045110
0x4cd41869,0.085993,0.189890,0.058218,0.138096
0x5351510d,0.007997,0.002201,0.008406,0.002320
0x64a30a4b,0.062008,0.057656,0.075407,0.066624
0x72b0d6a6,0.011389,0.004467,0.013906,0.005447
0x8d6f0c7b,0.037500,0.078177,0.043129,0.090624
0x97589cd1,0.044694,0.046209,0.046597,0.048927
0xa9bdf738,0.031311,0.073604,0.034839,0.079869


In [34]:
tec_qf_sme.view()

Row
    [0] Column(margin=(5, 10), name='TECQFSME')
        [0] StaticText(value='<b>TECQFSME</b>')
        [1] FloatSlider(end=4, name='Boost factor', value=1)
        [2] IntSlider(end=100000, name='Matching pool', step=5000, value=25000)
        [3] FloatInput(description='Summation of a..., disabled=True, name='Total donations', value=6142.066124660001)
        [4] FloatInput(description='Summation of m..., disabled=True, name='Total funding pool', value=31142.06612466)
    [1] Column
        [0] ParamMethod(method, _pane=HoloViews, defer_load=False)
        [1] ParamMethod(method, _pane=HoloViews, defer_load=False)

In [35]:
tec_qf_sme.view_expertise_signal()

:Bars   [projectId,Variable]   (value)